In [1]:
# coding:utf-8
import ast  # 用於將字串轉換成串列
import re

from flask import Flask
from flask import render_template, jsonify, redirect
from flask import request

# 用於隨機抽取回應（無答案時）
import random

####################################################################
# Setup environment value
####################################################################


import os
from dotenv import load_dotenv
load_dotenv()
distance_filter = float(os.getenv("distance_filter"))
flask_server_port = int(os.getenv("flask_server_port"))
# Webhook API
trello_request_limit = int(os.getenv("trello_request_limit"))
# Get Environment Value
anthropic_setup = os.getenv("anthropic_setup") in ["True", "true", "1"]
openai_setup = os.getenv("openai_setup") in ["True", "true", "1"]
roBERTa_setup = os.getenv("roBERTa_setup") in ["True", "true", "1"]
bert_setup = os.getenv("bert_setup") in ["True", "true", "1"]


# 讀取文字檔並轉換成串列
def txt_to_list(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.readlines()
            content = [line.strip() for line in content]
        return content
    except FileNotFoundError:
        print("找不到指定的檔案。請檢查檔案路徑是否正確。", file_path)
        return []
    except Exception as e:
        print("讀取檔案時發生錯誤：", e)
        return []


# 指定文字檔路徑
file_path = 'your_file.txt'  # 請替換成實際的檔案路徑
action_word_list = txt_to_list("./setting/action_word_list.txt")
not_found_msg_list = txt_to_list("./setting/not_found_msg_list.txt")
print("啟動詞與無資料罐頭訊息載入完成！")

啟動詞與無資料罐頭訊息載入完成！


In [2]:
####################################################################
# Process Module
####################################################################
# SBERT 編碼模組
# from modules.tools.process_words import embedding_sentence, process_sentence, generate_wordcloud
import modules.tools.process_words as process_words
# Milvus 向量搜尋與運算模組
# from modules.tools.vector_search import search_article_vector
import modules.tools.vector_search as vector_search
# MongoDB 文章搜尋模組
# from modules.tools.mongo_connector import find_article_info, add_trello_log
import modules.tools.mongo_connector as mongo_connector
# GPT 回答模組
# from modules.tools.answer_core import qa_by_anthropic, qa_by_openai, qa_by_RoBERTa, qa_by_bert
import modules.tools.answer_core as answer_core
# Trello 模組
# from modules.tools.trello_connector import updateDataToCard, addCommentToCard, addCommentWithPictureToCard, addCoverToCard
import modules.tools.trello_connector as trello_connector
# Vector 計算模組
import modules.tools.vector_calc as vector_calculation

c:\Users\vm\Anaconda3\envs\liang_trello_finder\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.
monpa_dict loaded
stop_word_list loaded
sbert_model loaded
Load Chinese stop words Done
Load Chinese font Done


08/17/2023 15:03:31 - DEBUG - httpx - load_ssl_context verify=True cert=None trust_env=True http2=False
08/17/2023 15:03:31 - DEBUG - httpx - load_verify_locations cafile='c:\\Users\\vm\\Anaconda3\\envs\\liang_trello_finder\\lib\\site-packages\\certifi\\cacert.pem'
08/17/2023 15:03:31 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): huggingface.co:443
08/17/2023 15:03:32 - DEBUG - urllib3.connectionpool - https://huggingface.co:443 "HEAD /uer/roberta-base-chinese-extractive-qa/resolve/main/config.json HTTP/1.1" 200 0
08/17/2023 15:03:33 - DEBUG - urllib3.connectionpool - https://huggingface.co:443 "HEAD /uer/roberta-base-chinese-extractive-qa/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
No CUDA runtime is found, using CUDA_HOME='C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.2'
08/17/2023 15:03:34 - DEBUG - urllib3.connectionpool - https://huggingface.co:443 "HEAD /NchuNLP/Chinese-Question-Answering/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
0

In [3]:
# 預處理 Milvus 回傳的資料
def process_milvus_result(req_array, sentence, anthropic_setup=False, openai_setup=False, roBERTa_setup=False, bert_setup=False):
    return_array = []
    for item in req_array:
        # IF distance < 設定值, break and return 相關性不足的文章
        if (item['distance'] < distance_filter):
            break

        # Use track_id to find Ariicle (Only one article)
        article_id = item['track_id']
        article = find_article_info(article_id)
        if article is not None:
            insert_data = {
                "id": item['id'],
                "distance": item['distance'],
                "title": article['title'],
                "url": article['url'],
                "content": article['content'],
            }
            # Use Meta's content to answer question by gpt
            if anthropic_setup:
                answer = qa_by_anthropic(article['content'], sentence)
                if (answer['state']):
                    insert_data["answer_by_anthropic"] = answer['value']
                else:
                    return {
                        "state": False,
                        "err_msg": answer['value'],
                        "show_msg": "anthropic GPT 模組失敗，請聯絡工程人員",
                        "error_code": 504,
                    }
            # Use Meta's content to answer question by openai
            if openai_setup:
                answer = qa_by_openai(article['content'], sentence)
                if (answer['state']):
                    insert_data["answer_by_openai"] = answer['value']
                else:
                    return {
                        "state": False,
                        "err_msg": answer['value'],
                        "show_msg": "openai GPT 模組失敗，請聯絡工程人員",
                        "error_code": 504,
                    }
            # Use Meta's content to answer question by RoBERTa
            if roBERTa_setup:
                answer = qa_by_RoBERTa(article['content'], sentence)
                if (answer['state']):
                    insert_data["answer_by_RoBERTa"] = answer['value']
                else:
                    return {
                        "state": False,
                        "err_msg": answer['value'],
                        "show_msg": "RoBERTa GPT 模組失敗，請聯絡工程人員",
                        "error_code": 504,
                    }
            # Use Meta's content to answer question by RoBERTa
            if bert_setup:
                answer = qa_by_bert(article['content'], sentence)
                if (answer['state']):
                    insert_data["answer_by_BERT"] = answer['value']
                else:
                    return {
                        "state": False,
                        "err_msg": answer['value'],
                        "show_msg": "BERT GPT 模組失敗，請聯絡工程人員",
                        "error_code": 504,
                    }
        else:
            print(article, "cannot find")

        return_array.append(insert_data)

    # End of Combine amd return the value
    return {
        "state": True,
        "result": return_array,
    }


# Webhook 處理流程
def process_webhook(data):
    try:
        # Convert Data
        print(data)
        user_input = data["user_input"]
        card_id = data["card_id"]
        checkIsTrello = data["is_trello"]
        fuzzy_sentence = process_sentence(user_input)

        if (checkIsTrello):
            try:
                updateDataToCard(card_id, {
                    "name": f"[進行中] {user_input}",
                    "desc": f"**關鍵字推薦：** \n\n{fuzzy_sentence}",
                })
            except Exception as exp:
                return {
                    "state": False,
                    "err_msg": str(exp),
                    "show_msg": "[updateDataToCard] 卡片更新失敗",
                }

        result_of_sentence = process_sentence_to_article_list(user_input,
                                                              setup={
                                                                  "limit": trello_request_limit,
                                                                  "anthropic_setup": anthropic_setup,
                                                                  "openai_setup": openai_setup,
                                                                  "roBERTa_setup": roBERTa_setup,
                                                                  "bert_setup": bert_setup,
                                                              })

        if (result_of_sentence['state']):
            # Add Comment
            if (len(result_of_sentence['result']) == 0):
                # Not Find Data so return random message
                commit_msg = random.choice(not_found_msg_list)

                if (checkIsTrello):
                    try:
                        addCommentToCard(card_id, commit_msg)
                        addCoverToCard(
                            card_id, "./static/images/not_found.png")
                    except Exception as exp:
                        return {
                            "state": False,
                            "err_msg": str(exp),
                            "show_msg": "[addCommentToCard] 卡片更新失敗",
                        }
            else:
                # Define String to generate wordcloud
                wc_string = ""

                # Process Result (Send to trello, convert commitmsg)
                for item in reversed(result_of_sentence['result']):
                    # 設定文字雲累加文本(文章內容)
                    wc_string += item['content'] + " "

                    # 設定留言資訊（Answer Core）
                    commit_msg = f"參考資料：\n[{item['title']}]({item['url']})（{item['id']}）\n"
                    if (anthropic_setup):
                        commit_msg += f"參考回答 A ：\n{item['answer_by_anthropic']} \n"
                    if (openai_setup):
                        commit_msg += f"參考回答 C ：\n{item['answer_by_openai']} \n"
                    if (roBERTa_setup):
                        commit_msg += f"參考回答 RB ：\n{item['answer_by_RoBERTa']} \n"
                    if (bert_setup):
                        commit_msg += f"參考回答 B ：\n{item['answer_by_BERT']} \n"

                    if (checkIsTrello):
                        # Add Comment
                        try:
                            # 留言不包含圖片 v1
                            # addCommentToCard(card_id,commit_msg)
                            # 留言包含圖片 v2 up
                            addCommentWithPictureToCard(
                                card_id, f"https://raw.githubusercontent.com/tony13382/trello_helper_img_w/main/{item['id']}.png", commit_msg)
                        except Exception as exp:
                            return {
                                "state": False,
                                "err_msg": str(exp),
                                "show_msg": "[addCommentToCard] 留言失敗",
                            }

                # All Done
                try:
                    if (checkIsTrello):
                        # 產生文字雲
                        wc_img_path = generate_wordcloud(wc_string)
                        if (wc_img_path["state"]):
                            # 更新封面
                            addCoverToCard(card_id, wc_img_path["value"])
                        else:
                            return {
                                "state": False,
                                "err_msg": wc_img_path["value"],
                                "show_msg": "[generate_wordcloud] 文字雲產生失敗",
                            }
                except Exception as exp:
                    return {
                        "state": False,
                        "err_msg": str(exp),
                        "show_msg": "[addCoverToCard] 卡片更新失敗",
                    }

            return {
                "state": True,
                "show_msg": "留言成功",
                "more_info": {
                    "user_input": user_input,
                    "searchResult": result_of_sentence['result'],
                }
            }
        else:
            return {
                "state": False,
                "err_msg": result_of_sentence['err_msg'],
                "show_msg": "[result_of_sentence] " + result_of_sentence['show_msg'],
            }
    except Exception as exp:
        return {
            "state": False,
            "err_msg": str(exp),
            "show_msg": "[process_webhook] 資料處理失敗，請聯絡工程人員",
        }

# 驗證文本是否包含動作關鍵字


def check_action_word(input_string, action_word_list):
    for action_word in action_word_list:
        if action_word in input_string:
            return True
    return False

In [4]:
def get_article_index_list_precise(wordlist):
    precise_article_list = []
    for word in wordlist:
        precise_article_list.extend(mongo_connector.get_alist_by_kw(word))

    if len(precise_article_list) == 0:
        return []

    # precise_list > 0  有找到文章
    # 如果 article_id 相同則累加 score, 最後根據 score 降冪排序
    # 创建一个字典来存储article_id对应的累加score
    article_scores = {}

    # 计算累加score
    for item in precise_article_list:
        article_id = item["article_id"]
        score = item["score"]
        if article_id in article_scores:
            article_scores[article_id] += score
        else:
            article_scores[article_id] = score

    # 将字典转换为列表，按照score降序排序
    result_array = [{"article_id": article_id, "score": score} for article_id,
                    score in sorted(article_scores.items(), key=lambda x: x[1], reverse=True)]
    if len(result_array) > 20:
        result_array = result_array[:20]
    return result_array

In [30]:
def webhook_v2(user_input, card_id, checkIsTrello=False):

    # 定義需要留言的組件
    trello_commit = []
    wc_string = ""

    # 驗證文本是否包含動作關鍵字
    if check_action_word(user_input, action_word_list):
        # 文本包含動作關鍵字
        # 開始清理文字
        if checkIsTrello is True:
            trello_connector.updateDataToCard(card_id, {
                "name": f"[進行中] {user_input}",
            })

        print("開始清理文字")
        sliced_word_list = process_words.process_sentence(
            user_input, process_injectionword_setup=False).split()  # 只清洗文字
        print(sliced_word_list)
        print("開始精準搜尋")

        # 開始精準搜尋
        precise_result = get_article_index_list_precise(sliced_word_list)

        comment_precise_msg = "**精準搜尋結果：** \n --- \n\n"
        if len(precise_result) == 0:
            # 精準搜尋沒有結果
            comment_precise_msg += f"- 精準搜尋沒有結果 \n"
            print("精準搜尋沒有結果")
        else:
            result = []
            print(f"精準搜尋共 {len(precise_result)} 筆資料")

            if len(precise_result) > 20:
                print("精準搜尋結果過多，僅顯示前 20 筆")
                precise_result = precise_result[:20]

            counter = 0
            for article_object in precise_result:
                article_info = mongo_connector.find_article_info(
                    str(article_object["article_id"]))
                if article_info is not None and len(article_info) > 0:
                    counter += 1
                    comment_precise_msg += f"{counter}. [{article_info['title']}]({article_info['url']}) \n"
                    result.append({
                        "article_id": article_object["article_id"],
                        "score": article_object["score"],
                        "title": article_info["title"],
                        "url": article_info["url"],
                        "content": article_info["content"],
                    })
                    wc_string += f'{article_info["content"]} '
            precise_result_index = [x["article_id"] for x in precise_result]

            trello_commit.append(comment_precise_msg)
            # print(comment_precise_msg)
            print("精準搜尋階段完成")

        #################################################################################
        #################################################################################

        # 開始文章文本模糊搜索
        print("開始文章文本模糊搜索")

        fuzzy_search_alist = []
        comment_fuzzy_msg = "**相似搜尋結果：**\n --- \n\n"
        # 處理句子
        orginal_sentence = process_words.process_sentence(
            user_input, process_injectionword_setup=False)  # 只清洗文字
        injected_sentence = process_words.process_sentence(
            user_input)  # 清洗文字並且進行相似詞搜尋

        # 轉換成向量
        o_vector = process_words.embedding_sentence(orginal_sentence)
        f_vector = process_words.embedding_sentence(injected_sentence)
        q_vector = f_vector

        # 權重計算與調整
        orginal_weight = 1
        fuzzy_weight = 3

        if (o_vector['state'] and f_vector['state']):
            q_vector["value"] = vector_calculation.calc_array_mean(
                set=[{
                    "array": o_vector['value'],
                    "weight": orginal_weight,
                }, {
                    "array": f_vector['value'],
                    "weight": fuzzy_weight,
                }],
                len_array=768
            )
        else:
            print("向量轉換失敗")

        if q_vector['state']:
            print("向量計算成功")
            fuzzy_search_result = vector_search.search_article_vector(
                q_vector["value"])

        if fuzzy_search_result['state']:
            print("相似文章搜尋成功")
            fuzzy_search_alist = [
                x["id"] for x in fuzzy_search_result["value"] if x["distance"] > 2.75]

        if len(fuzzy_search_alist) > 0:
            print(f"相似文章搜尋共 {len(fuzzy_search_alist)} 筆資料")
            if len(fuzzy_search_alist) > 20:
                print("相似文章搜尋結果過多，僅顯示前 20 筆")
                fuzzy_search_alist = fuzzy_search_alist[:20]
            # 內容輸出
            counter = 0
            for a_id in fuzzy_search_alist:
                a_info = mongo_connector.find_article_info(a_id)
                if a_info is None:
                    continue
                else:
                    counter += 1
                    comment_fuzzy_msg += f"{counter}. [{a_info['title']}]({a_info['url']}) \n"
                    result.append({
                        "article_id": a_id,
                        "title": a_info["title"],
                        "url": a_info["url"],
                        "content": a_info["content"],
                    })
                    wc_string += f'{a_info["content"]} '

        print("文章文本模糊搜索結束")

        trello_commit.append(comment_fuzzy_msg)

        #################################################################################
        #################################################################################

        # 開始文本注入搜尋
        print("開始文本注入搜尋")
        comment_injected_msg = "**創意搜尋結果：**\n --- \n\n"
        # 開始注入
        injected_sentence = process_words.process_sentence(
            user_input, process_injectionword_setup=True)
        print(f"注入文本：{user_input} \n-> {injected_sentence}")
        comment_injected_msg += f"{injected_sentence} \n---\n"
        # 向量化注入文本
        injected_vector = process_words.embedding_sentence(injected_sentence)
        if injected_vector == False:
            print("文本注入向量化失敗")
        else:
            # 取得相關關鍵字列表 by Milvus
            kwlist_by_m = vector_search.search_keyword_vector(
                injected_vector["value"])

        if kwlist_by_m["state"] == False:
            print("文本注入搜尋失敗")
        else:
            # 取得相關關鍵字編號
            kwlist_index = [x["track_id"] for x in kwlist_by_m["value"]]
            # 取得相關關鍵字文章列表
            inject_alist = mongo_connector.get_alist_by_klist(kwlist_index)
            inject_alist = [x for x in inject_alist if x["total_score"] > 0.05]

            if len(inject_alist) > 0:
                print(f"文本注入搜尋共 {len(inject_alist)} 筆資料")
                if len(inject_alist) > 20:
                    print("文本注入搜尋結果過多，僅顯示前 20 筆")
                    inject_alist = inject_alist[:20]
                # 內容輸出
                counter = 0
                for a_id in inject_alist:
                    a_info = mongo_connector.find_article_info(a_id["_id"])
                    if a_info is None:
                        continue
                    else:
                        counter += 1
                        comment_injected_msg += f"{counter}. [{a_info['title']}]({a_info['url']}) \n"
                        result.append({
                            "article_id": a_id["_id"],
                            "score": a_id["total_score"],
                            "title": a_info["title"],
                            "url": a_info["url"],
                            "content": a_info["content"],
                        })
                        wc_string += f'{a_info["content"]} '
            else:
                comment_injected_msg += f"- 精準搜尋沒有結果 \n"
                print("文本注入搜尋結果為空")

        trello_commit.append(comment_injected_msg)

        #################################################################################
        #################################################################################

        if checkIsTrello is True:
            # 產生文字雲
            wc_img_path = process_words.generate_wordcloud(
                wc_string, f"{card_id}_綜合文字雲.png")
            if (wc_img_path["state"]):
                # 更新封面
                print("WordCloud 圖片產生成功")
                trello_connector.addCoverToCard(card_id, wc_img_path["value"])

            for commit_msg in reversed(trello_commit):
                trello_connector.addCommentToCard(card_id, commit_msg)

            trello_connector.updateDataToCard(card_id, {
                "name": f"[已完成] {user_input}",
            })
        print("搜索結束")

In [32]:
wc_string

NameError: name 'wc_string' is not defined

In [14]:
wc_img_path_injected = process_words.generate_wordcloud(
    "123 1213131 122 22gsfs", f"創意搜尋文字雲")

08/17/2023 15:52:44 - DEBUG - matplotlib.pyplot - Loaded backend Agg version v2.2.


In [31]:
webhook_v2("虎克定律是什麼？", "64dce092ee0aa311c0da2d38", checkIsTrello=True)

08/17/2023 00:08:16 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): api.trello.com:443


08/17/2023 00:08:17 - DEBUG - urllib3.connectionpool - https://api.trello.com:443 "PUT /1/cards/64dce092ee0aa311c0da2d38?key=d9dc5e46916ad23019fd34a128895b76&token=ATTA1a28707482d0c918707e4faad9cba87db844d23422107543bcfafd9eeaed33cc68D03287&name=%5B%E9%80%B2%E8%A1%8C%E4%B8%AD%5D+%E8%99%8E%E5%85%8B%E5%AE%9A%E5%BE%8B%E6%98%AF%E4%BB%80%E9%BA%BC%EF%BC%9F HTTP/1.1" 200 1198


{
    "badges": {
        "attachments": 0,
        "attachmentsByType": {
            "trello": {
                "board": 0,
                "card": 0
            }
        },
        "checkItems": 0,
        "checkItemsChecked": 0,
        "checkItemsEarliestDue": null,
        "comments": 3,
        "description": false,
        "due": null,
        "dueComplete": false,
        "fogbugz": "",
        "location": false,
        "start": null,
        "subscribed": false,
        "viewingMemberVoted": false,
        "votes": 0
    },
    "cardRole": null,
    "checkItemStates": [],
    "closed": false,
    "cover": {
        "brightness": "dark",
        "color": null,
        "idAttachment": null,
        "idPlugin": null,
        "idUploadedBackground": null,
        "size": "normal"
    },
    "dateLastActivity": "2023-08-16T16:08:17.265Z",
    "desc": "",
    "descData": {
        "emoji": {}
    },
    "due": null,
    "dueComplete": false,
    "dueReminder": null,
    "email":

Batches: 100%|██████████| 1/1 [00:00<00:00, 22.10it/s]


向量計算成功
相似文章搜尋成功
相似文章搜尋共 6 筆資料
文章文本模糊搜索結束
開始文本注入搜尋
注入文本：虎克定律是什麼？ 
-> 虎克 : 虎克定律 Hooke’s law 應力 應變 運動與力 彈簧



...

{
    "appCreator": {
        "icon": {
            "url": "https://tony13382.github.io/2023_TrelloFinder_Img/botIcon.svg"
        },
        "id": "64afa44daa99a34b250a3622"
    },
    "data": {
        "board": {
            "id": "64b16c389a83b89a449b689e",
            "name": "Test For Lab",
            "shortLink": "nenizuaM"
        },
        "card": {
            "id": "64dce092ee0aa311c0da2d38",
            "idShort": 295,
            "name": "[\u9032\u884c\u4e2d] \u864e\u514b\u5b9a\u5f8b\u662f\u4ec0\u9ebc\uff1f",
            "shortLink": "kH9P2VxU"
        },
        "list": {
            "id": "64cce3d284f2af49642af6ab",
            "name": "T4"
        },
        "text": "**\u76f8\u4f3c\u641c\u5c0b\u7d50\u679c\uff1a**\n --- \n\n1. [\u864e\u514b\u5b9a\u5f8b Hooke\u2019s law](https://highscope.ch.ntu.edu.tw/wordpress/?p=46203) \n2. [\u725b\u9813\u7b2c\u4e09\u904b\u52d5\u5b9a\u5f8b\uff08Newton\u2019s Third Laws of Motion\uff09](https://highscope.ch.ntu.edu.tw/wordpress/?p=1873

08/17/2023 00:08:27 - DEBUG - urllib3.connectionpool - https://api.trello.com:443 "POST /1/cards/64dce092ee0aa311c0da2d38/actions/comments?text=%2A%2A%E7%B2%BE%E6%BA%96%E6%90%9C%E5%B0%8B%E7%B5%90%E6%9E%9C%EF%BC%9A%2A%2A+%0A+---+%0A%0A1.+%5B%E8%99%8E%E5%85%8B+%28Robert+Hooke%29%5D%28https%3A%2F%2Fhighscope.ch.ntu.edu.tw%2Fwordpress%2F%3Fp%3D34511%29+%0A2.+%5B%E8%99%8E%E5%85%8B%E5%AE%9A%E5%BE%8B+Hooke%E2%80%99s+law%5D%28https%3A%2F%2Fhighscope.ch.ntu.edu.tw%2Fwordpress%2F%3Fp%3D46203%29+%0A3.+%5B%E8%B7%B3%E9%AB%98%E7%9A%84%E4%B8%96%E7%95%8C%E5%86%A0%E8%BB%8D%EF%BD%9E%E6%B2%AB%E8%9F%AC%5D%28https%3A%2F%2Fhighscope.ch.ntu.edu.tw%2Fwordpress%2F%3Fp%3D53339%29+%0A4.+%5B1665%E5%B9%B41%E6%9C%88%EF%BC%9A%E8%99%8E%E5%85%8B%E3%80%8A%E5%BE%AE%E7%89%A9%E5%9C%96%E8%A7%A3%E3%80%8B%E7%9A%84%E5%87%BA%E7%89%88%5D%28https%3A%2F%2Fhighscope.ch.ntu.edu.tw%2Fwordpress%2F%3Fp%3D1476%29+%0A5.+%5B%E5%BD%88%E6%80%A7%E5%B8%B8%E6%95%B8+Spring+Constant%5D%28https%3A%2F%2Fhighscope.ch.ntu.edu.tw%2Fwordpress%2F%3Fp%

{
    "appCreator": {
        "icon": {
            "url": "https://tony13382.github.io/2023_TrelloFinder_Img/botIcon.svg"
        },
        "id": "64afa44daa99a34b250a3622"
    },
    "data": {
        "board": {
            "id": "64b16c389a83b89a449b689e",
            "name": "Test For Lab",
            "shortLink": "nenizuaM"
        },
        "card": {
            "id": "64dce092ee0aa311c0da2d38",
            "idShort": 295,
            "name": "[\u9032\u884c\u4e2d] \u864e\u514b\u5b9a\u5f8b\u662f\u4ec0\u9ebc\uff1f",
            "shortLink": "kH9P2VxU"
        },
        "list": {
            "id": "64cce3d284f2af49642af6ab",
            "name": "T4"
        },
        "text": "**\u7cbe\u6e96\u641c\u5c0b\u7d50\u679c\uff1a** \n --- \n\n1. [\u864e\u514b (Robert Hooke)](https://highscope.ch.ntu.edu.tw/wordpress/?p=34511) \n2. [\u864e\u514b\u5b9a\u5f8b Hooke\u2019s law](https://highscope.ch.ntu.edu.tw/wordpress/?p=46203) \n3. [\u8df3\u9ad8\u7684\u4e16\u754c\u51a0\u8ecd\uff5e\u6cab\u8

08/17/2023 00:08:28 - DEBUG - urllib3.connectionpool - https://api.trello.com:443 "PUT /1/cards/64dce092ee0aa311c0da2d38?key=d9dc5e46916ad23019fd34a128895b76&token=ATTA1a28707482d0c918707e4faad9cba87db844d23422107543bcfafd9eeaed33cc68D03287&name=%5B%E5%B7%B2%E5%AE%8C%E6%88%90%5D+%E8%99%8E%E5%85%8B%E5%AE%9A%E5%BE%8B%E6%98%AF%E4%BB%80%E9%BA%BC%EF%BC%9F HTTP/1.1" 200 1198


{
    "badges": {
        "attachments": 0,
        "attachmentsByType": {
            "trello": {
                "board": 0,
                "card": 0
            }
        },
        "checkItems": 0,
        "checkItemsChecked": 0,
        "checkItemsEarliestDue": null,
        "comments": 6,
        "description": false,
        "due": null,
        "dueComplete": false,
        "fogbugz": "",
        "location": false,
        "start": null,
        "subscribed": false,
        "viewingMemberVoted": false,
        "votes": 0
    },
    "cardRole": null,
    "checkItemStates": [],
    "closed": false,
    "cover": {
        "brightness": "dark",
        "color": null,
        "idAttachment": null,
        "idPlugin": null,
        "idUploadedBackground": null,
        "size": "normal"
    },
    "dateLastActivity": "2023-08-16T16:08:28.327Z",
    "desc": "",
    "descData": {
        "emoji": {}
    },
    "due": null,
    "dueComplete": false,
    "dueReminder": null,
    "email":

In [25]:
fuzzy_search_alist

['2705', '2836', '3462', '3633', '3296', '3539']

In [21]:
user_input = "虎克定律是什麼？"   # 使用者輸入的文字
card_id = "64dce092ee0aa311c0da2d38"
checkIsTrello = True

# 定義需要留言的組件
trello_commit = []

# 驗證文本是否包含動作關鍵字
if check_action_word(user_input, action_word_list):
    # 文本包含動作關鍵字
    # 開始清理文字
    if checkIsTrello is True:
        trello_connector.updateDataToCard(card_id, {
            "name": f"[進行中] {user_input}",
        })

    print("開始清理文字")
    sliced_word_list = process_words.process_sentence(
        user_input, process_injectionword_setup=False).split()  # 只清洗文字
    print(sliced_word_list)
    print("開始精準搜尋")

    # 開始精準搜尋
    precise_result = get_article_index_list_precise(sliced_word_list)

    comment_precise_msg = "**精準搜尋結果：** \n --- \n\n"
    if len(precise_result) == 0:
        # 精準搜尋沒有結果
        comment_precise_msg += f"- 精準搜尋沒有結果 \n"
        print("精準搜尋沒有結果")
    else:
        result = []
        print(f"精準搜尋共 {len(precise_result)} 筆資料")

        if len(precise_result) > 20:
            print("精準搜尋結果過多，僅顯示前 20 筆")
            precise_result = precise_result[:20]

        counter = 0
        for article_object in precise_result:
            article_info = mongo_connector.find_article_info(
                str(article_object["article_id"]))
            if article_info is not None and len(article_info) > 0:
                counter += 1
                comment_precise_msg = comment_precise_msg + \
                    f"{counter}. [{article_info['title']}]({article_info['url']}) \n"
                result.append({
                    "article_id": article_object["article_id"],
                    "score": article_object["score"],
                    "title": article_info["title"],
                    "url": article_info["url"],
                    "content": article_info["content"],
                })
        precise_result_index = [x["article_id"] for x in precise_result]

        trello_commit.append(comment_precise_msg)
        # print(comment_precise_msg)
        print("精準搜尋階段完成")

    #################################################################################
    #################################################################################

    # 開始文章文本模糊搜索
    print("開始文章文本模糊搜索")

    fuzzy_search_alist = []
    comment_fuzzy_msg = "**相似搜尋結果：**\n --- \n\n"
    # 處理句子
    orginal_sentence = process_words.process_sentence(
        user_input, process_injectionword_setup=False)  # 只清洗文字
    injected_sentence = process_words.process_sentence(
        user_input)  # 清洗文字並且進行相似詞搜尋

    # 轉換成向量
    o_vector = process_words.embedding_sentence(orginal_sentence)
    f_vector = process_words.embedding_sentence(injected_sentence)
    q_vector = f_vector

    # 權重計算與調整
    orginal_weight = 1
    fuzzy_weight = 3

    if (o_vector['state'] and f_vector['state']):
        q_vector["value"] = vector_calculation.calc_array_mean(
            set=[{
                "array": o_vector['value'],
                "weight": orginal_weight,
            }, {
                "array": f_vector['value'],
                "weight": fuzzy_weight,
            }],
            len_array=768
        )
    else:
        print("向量轉換失敗")

    if q_vector['state']:
        print("向量計算成功")
        fuzzy_search_result = vector_search.search_article_vector(
            q_vector["value"])

    if fuzzy_search_result['state']:
        print("相似文章搜尋成功")
        fuzzy_search_alist = [
            x["id"] for x in fuzzy_search_result["value"] if x["distance"] > 2.75]

    if len(fuzzy_search_alist) > 0:
        print(f"相似文章搜尋共 {len(fuzzy_search_alist)} 筆資料")
        if len(fuzzy_search_alist) > 20:
            print("相似文章搜尋結果過多，僅顯示前 20 筆")
            fuzzy_search_alist = fuzzy_search_alist[:20]
        # 內容輸出
        counter = 0
        for a_id in fuzzy_search_alist:
            a_info = mongo_connector.find_article_info(a_id)
            if a_info is None:
                continue
            else:
                counter += 1
                comment_fuzzy_msg += f"{counter}. [{a_info['title']}]({a_info['url']}) \n"
                result.append({
                    "article_id": article_object["article_id"],
                    "score": article_object["score"],
                    "title": article_info["title"],
                    "url": article_info["url"],
                    "content": article_info["content"],
                })

    print("文章文本模糊搜索結束")

    trello_commit.append(comment_fuzzy_msg)

    #################################################################################
    #################################################################################

    # 開始文本注入搜尋
    print("開始文本注入搜尋")
    comment_injected_msg = "**創意搜尋結果：**\n --- \n\n"
    # 開始注入
    injected_sentence = process_words.process_sentence(
        user_input, process_injectionword_setup=True)
    print(f"注入文本：{user_input} \n-> {injected_sentence}")
    comment_injected_msg += f"{injected_sentence} \n---\n"
    # 向量化注入文本
    injected_vector = process_words.embedding_sentence(injected_sentence)
    if injected_vector == False:
        print("文本注入向量化失敗")
    else:
        # 取得相關關鍵字列表 by Milvus
        kwlist_by_m = vector_search.search_keyword_vector(
            injected_vector["value"])

    if kwlist_by_m["state"] == False:
        print("文本注入搜尋失敗")
    else:
        # 取得相關關鍵字編號
        kwlist_index = [x["track_id"] for x in kwlist_by_m["value"]]
        # 取得相關關鍵字文章列表
        inject_alist = mongo_connector.get_alist_by_klist(kwlist_index)
        inject_alist = [x for x in inject_alist if x["total_score"] > 0.05]

        if len(inject_alist) > 0:
            print(f"文本注入搜尋共 {len(inject_alist)} 筆資料")
            if len(inject_alist) > 20:
                print("文本注入搜尋結果過多，僅顯示前 20 筆")
                inject_alist = inject_alist[:20]
            # 內容輸出
            counter = 0
            for a_id in inject_alist:
                a_info = mongo_connector.find_article_info(a_id["_id"])
                if a_info is None:
                    continue
                else:
                    counter += 1
                    comment_injected_msg += f"{counter}. [{a_info['title']}]({a_info['url']}) \n"
                    result.append({
                        "article_id": a_id["_id"],
                        "score": a_id["total_score"],
                        "title": a_info["title"],
                        "url": a_info["url"],
                        "content": a_info["content"],
                    })
        else:
            comment_injected_msg += f"- 精準搜尋沒有結果 \n"
            print("文本注入搜尋結果為空")

    trello_commit.append(comment_injected_msg)

    #################################################################################
    #################################################################################

    if checkIsTrello is True:
        for commit_msg in reversed(trello_commit):
            trello_connector.addCommentToCard(card_id, commit_msg)

        trello_connector.updateDataToCard(card_id, {
            "name": f"[已完成] {user_input}",
        })
    print("搜索結束")

08/16/2023 23:47:01 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): api.trello.com:443
08/16/2023 23:47:02 - DEBUG - urllib3.connectionpool - https://api.trello.com:443 "PUT /1/cards/64dce092ee0aa311c0da2d38?key=d9dc5e46916ad23019fd34a128895b76&token=ATTA1a28707482d0c918707e4faad9cba87db844d23422107543bcfafd9eeaed33cc68D03287&name=%5B%E9%80%B2%E8%A1%8C%E4%B8%AD%5D+%E8%99%8E%E5%85%8B%E5%AE%9A%E5%BE%8B%E6%98%AF%E4%BB%80%E9%BA%BC%EF%BC%9F HTTP/1.1" 200 1198


{
    "badges": {
        "attachments": 0,
        "attachmentsByType": {
            "trello": {
                "board": 0,
                "card": 0
            }
        },
        "checkItems": 0,
        "checkItemsChecked": 0,
        "checkItemsEarliestDue": null,
        "comments": 0,
        "description": false,
        "due": null,
        "dueComplete": false,
        "fogbugz": "",
        "location": false,
        "start": null,
        "subscribed": false,
        "viewingMemberVoted": false,
        "votes": 0
    },
    "cardRole": null,
    "checkItemStates": [],
    "closed": false,
    "cover": {
        "brightness": "dark",
        "color": null,
        "idAttachment": null,
        "idPlugin": null,
        "idUploadedBackground": null,
        "size": "normal"
    },
    "dateLastActivity": "2023-08-16T15:47:02.358Z",
    "desc": "",
    "descData": {
        "emoji": {}
    },
    "due": null,
    "dueComplete": false,
    "dueReminder": null,
    "email":

Batches: 100%|██████████| 1/1 [00:00<00:00, 24.01it/s]


向量計算成功
相似文章搜尋成功
相似文章搜尋共 6 筆資料
文章文本模糊搜索結束
開始文本注入搜尋
注入文本：虎克定律是什麼？ 
-> 虎克 : 虎克定律 Hooke’s law 應力 應變 運動與力 彈簧



Batches: 100%|██████████| 1/1 [00:00<00:00, 21.13it/s]


...

{
    "appCreator": {
        "icon": {
            "url": "https://tony13382.github.io/2023_TrelloFinder_Img/botIcon.svg"
        },
        "id": "64afa44daa99a34b250a3622"
    },
    "data": {
        "board": {
            "id": "64b16c389a83b89a449b689e",
            "name": "Test For Lab",
            "shortLink": "nenizuaM"
        },
        "card": {
            "id": "64dce092ee0aa311c0da2d38",
            "idShort": 295,
            "name": "[\u9032\u884c\u4e2d] \u864e\u514b\u5b9a\u5f8b\u662f\u4ec0\u9ebc\uff1f",
            "shortLink": "kH9P2VxU"
        },
        "list": {
            "id": "64cce3d284f2af49642af6ab",
            "name": "T4"
        },
        "text": "**\u76f8\u4f3c\u641c\u5c0b\u7d50\u679c\uff1a**\n --- \n\n1. [\u864e\u514b\u5b9a\u5f8b Hooke\u2019s law](https://highscope.ch.ntu.edu.tw/wordpress/?p=46203) \n2. [\u725b\u9813\u7b2c\u4e09\u904b\u52d5\u5b9a\u5f8b\uff08Newton\u2019s Third Laws of Motion\uff09](https://highscope.ch.ntu.edu.tw/wordpress/?p=1873

08/16/2023 23:47:08 - DEBUG - urllib3.connectionpool - https://api.trello.com:443 "POST /1/cards/64dce092ee0aa311c0da2d38/actions/comments?text=%2A%2A%E7%B2%BE%E6%BA%96%E6%90%9C%E5%B0%8B%E7%B5%90%E6%9E%9C%EF%BC%9A%2A%2A+%0A+---+%0A%0A1.+%5B%E8%99%8E%E5%85%8B+%28Robert+Hooke%29%5D%28https%3A%2F%2Fhighscope.ch.ntu.edu.tw%2Fwordpress%2F%3Fp%3D34511%29+%0A2.+%5B%E8%99%8E%E5%85%8B%E5%AE%9A%E5%BE%8B+Hooke%E2%80%99s+law%5D%28https%3A%2F%2Fhighscope.ch.ntu.edu.tw%2Fwordpress%2F%3Fp%3D46203%29+%0A3.+%5B%E8%B7%B3%E9%AB%98%E7%9A%84%E4%B8%96%E7%95%8C%E5%86%A0%E8%BB%8D%EF%BD%9E%E6%B2%AB%E8%9F%AC%5D%28https%3A%2F%2Fhighscope.ch.ntu.edu.tw%2Fwordpress%2F%3Fp%3D53339%29+%0A4.+%5B1665%E5%B9%B41%E6%9C%88%EF%BC%9A%E8%99%8E%E5%85%8B%E3%80%8A%E5%BE%AE%E7%89%A9%E5%9C%96%E8%A7%A3%E3%80%8B%E7%9A%84%E5%87%BA%E7%89%88%5D%28https%3A%2F%2Fhighscope.ch.ntu.edu.tw%2Fwordpress%2F%3Fp%3D1476%29+%0A5.+%5B%E5%BD%88%E6%80%A7%E5%B8%B8%E6%95%B8+Spring+Constant%5D%28https%3A%2F%2Fhighscope.ch.ntu.edu.tw%2Fwordpress%2F%3Fp%

{
    "appCreator": {
        "icon": {
            "url": "https://tony13382.github.io/2023_TrelloFinder_Img/botIcon.svg"
        },
        "id": "64afa44daa99a34b250a3622"
    },
    "data": {
        "board": {
            "id": "64b16c389a83b89a449b689e",
            "name": "Test For Lab",
            "shortLink": "nenizuaM"
        },
        "card": {
            "id": "64dce092ee0aa311c0da2d38",
            "idShort": 295,
            "name": "[\u9032\u884c\u4e2d] \u864e\u514b\u5b9a\u5f8b\u662f\u4ec0\u9ebc\uff1f",
            "shortLink": "kH9P2VxU"
        },
        "list": {
            "id": "64cce3d284f2af49642af6ab",
            "name": "T4"
        },
        "text": "**\u7cbe\u6e96\u641c\u5c0b\u7d50\u679c\uff1a** \n --- \n\n1. [\u864e\u514b (Robert Hooke)](https://highscope.ch.ntu.edu.tw/wordpress/?p=34511) \n2. [\u864e\u514b\u5b9a\u5f8b Hooke\u2019s law](https://highscope.ch.ntu.edu.tw/wordpress/?p=46203) \n3. [\u8df3\u9ad8\u7684\u4e16\u754c\u51a0\u8ecd\uff5e\u6cab\u8

08/16/2023 23:47:09 - DEBUG - urllib3.connectionpool - https://api.trello.com:443 "PUT /1/cards/64dce092ee0aa311c0da2d38?key=d9dc5e46916ad23019fd34a128895b76&token=ATTA1a28707482d0c918707e4faad9cba87db844d23422107543bcfafd9eeaed33cc68D03287&name=%5B%E5%B7%B2%E5%AE%8C%E6%88%90%5D+%E8%99%8E%E5%85%8B%E5%AE%9A%E5%BE%8B%E6%98%AF%E4%BB%80%E9%BA%BC%EF%BC%9F HTTP/1.1" 200 1198


{
    "badges": {
        "attachments": 0,
        "attachmentsByType": {
            "trello": {
                "board": 0,
                "card": 0
            }
        },
        "checkItems": 0,
        "checkItemsChecked": 0,
        "checkItemsEarliestDue": null,
        "comments": 3,
        "description": false,
        "due": null,
        "dueComplete": false,
        "fogbugz": "",
        "location": false,
        "start": null,
        "subscribed": false,
        "viewingMemberVoted": false,
        "votes": 0
    },
    "cardRole": null,
    "checkItemStates": [],
    "closed": false,
    "cover": {
        "brightness": "dark",
        "color": null,
        "idAttachment": null,
        "idPlugin": null,
        "idUploadedBackground": null,
        "size": "normal"
    },
    "dateLastActivity": "2023-08-16T15:47:08.990Z",
    "desc": "",
    "descData": {
        "emoji": {}
    },
    "due": null,
    "dueComplete": false,
    "dueReminder": null,
    "email":

---


In [8]:
# 取得其他關鍵字
tf_kw_list = mongo_connector.get_other_keyword_by_alist(precise_result_index)
# 分數大於0.5的才要
tf_kw_list = [x["_id"] for x in tf_kw_list if x["total_score"] > 0.5]
if len(tf_kw_list) > 20:
    tf_kw_list = tf_kw_list[:20]

# 取得其他關鍵字的文章
tf_alist = mongo_connector.get_alist_by_klist(tf_kw_list)

In [10]:
mongo_connector.convert_klistid_to_info(tf_kw_list)

['資料',
 '網路',
 '密碼',
 '觀測',
 '氣象',
 '資訊',
 '成本',
 '生物',
 '學習',
 '視覺化',
 'AirPods',
 '臉書',
 '中',
 '耳機',
 '多樣性',
 '研究',
 '隱私',
 '功能',
 '更',
 '大數據']

In [26]:
tf_alist = [
    {
        "a_id": item["_id"],
        "score": item["total_score"],
        "title": mongo_connector.find_article_info(str(item["_id"]))["title"]
    }
    for item in tf_alist
    if item["total_score"] > 0.5
]

In [27]:
tf_alist

[{'a_id': 8032, 'score': 11.558171745152356, 'title': '【愛知目標】全球與台灣生物多樣性的當前進展'},
 {'a_id': 8027,
  'score': 8.940366972477063,
  'title': '生物多樣性是什麼？ 近7成民眾聽過 僅3成正確理解'},
 {'a_id': 8023,
  'score': 8.893470790378005,
  'title': '生物多樣性就在身邊 搭建主流化夥伴關係 救石虎你也做得到'},
 {'a_id': 8038,
  'score': 7.1104252400548695,
  'title': '【愛知目標】全球生物多樣性的當前進展與改善要務（上）'},
 {'a_id': 6694, 'score': 6.257555847568988, 'title': '人．自然．保育真諦'},
 {'a_id': 841,
  'score': 6.076487252124647,
  'title': '老天啊～我真是搞不懂你！ (一) 氣象觀測（Meteorological Observation）'},
 {'a_id': 1159,
  'score': 5.9921259842519685,
  'title': '生物多樣性（Biodiversity）與生物多樣性公約 （Convention on Biological Diversity; CBD)'},
 {'a_id': 8037,
  'score': 5.631663974151858,
  'title': '【愛知目標】全球生物多樣性的當前進展與改善要務（下）'},
 {'a_id': 8015, 'score': 5.599783080260304, 'title': '挽救生物多樣性 城市不容缺席'},
 {'a_id': 8045, 'score': 5.110032362459547, 'title': '淺談生物多樣性與愛知目標'},
 {'a_id': 12054,
  'score': 4.965311004784689,
  'title': '非洲的基因資料庫爭議，開放科學與限制使用授權一定衝突嗎？'},
 {'a_id': 9326,
  'score

In [12]:
from collections import defaultdict
from operator import itemgetter

# 創建一個 defaultdict 來儲存 keyword_id 對應的總分數
keyword_scores = defaultdict(float)

# 找到 precise_result_index 中的所有對應的 keyword_record
for article_id in precise_result_index:
    keyword_records = mongo_connector.find_keyrecord_by_aid(article_id)

    # 計算每個 keyword_id 的總分數
    for record in keyword_records:
        keyword_id = record['keyword_id']
        score = record['score']
        keyword_scores[keyword_id] += score

# 將 keyword_scores 字典轉換成列表，並按照 score 進行降冪排序
sorted_keywords = sorted(keyword_scores.items(),
                         key=itemgetter(1), reverse=True)

# 取得前 10 筆 keyword_id
top_keywords = [keyword_id for keyword_id, _ in sorted_keywords[:10]]

In [9]:
sorted_keywords = [keyword_id, score in sorted(
    keyword_scores.items(), key=itemgetter(1), reverse=True)]

In [ ]:
sorted_keywords

[2759, False]

In [10]:
get_article_index_list_precise_result

[{'article_id': 8367, 'score': 4.1191489361702125},
 {'article_id': 354, 'score': 3.2329896907216495},
 {'article_id': 192, 'score': 2.9932249322493223},
 {'article_id': 34, 'score': 1.5986842105263157},
 {'article_id': 11689, 'score': 1.5361484325015997},
 {'article_id': 364, 'score': 1.4853168469860896},
 {'article_id': 189, 'score': 1.381294964028777},
 {'article_id': 161, 'score': 1.2943820224719103},
 {'article_id': 73, 'score': 1.2655502392344498},
 {'article_id': 9328, 'score': 1.235042735042735},
 {'article_id': 835, 'score': 1.2195945945945945},
 {'article_id': 8036, 'score': 1.208086785009862},
 {'article_id': 841, 'score': 1.13314447592068},
 {'article_id': 15108, 'score': 1.1198402130492675},
 {'article_id': 406, 'score': 1.0997506234413965},
 {'article_id': 169, 'score': 1.0982857142857143},
 {'article_id': 160, 'score': 1.0903225806451613},
 {'article_id': 163, 'score': 1.05125},
 {'article_id': 7431, 'score': 1.0413385826771655},
 {'article_id': 9326, 'score': 1.01185770